# Rule merging 

---

## 1. Importing necessary modules

In [36]:
from utils.N01_agent_setup import (
    # Fact classes for our rule engine
    TelemetryFact,
    DegradationFact,
    GapFact,
    RadioFact,
    RaceStatusFact,
    StrategyRecommendation,
    F1StrategyEngine,
    
    # Utility functions for data transformation
    transform_tire_predictions,
    load_tire_predictions,
    transform_lap_time_predictions,
    load_lap_time_predictions,
    transform_radio_analysis,
    process_radio_message,
    transform_gap_data_with_consistency,
    load_gap_data,
    calculate_gap_consistency
)

# Import the rule engines from each domain
# Tire degradation rules
from utils.N02_degradation_time_rules import F1DegradationRules

# Lap time prediction rules
from utils.N03_lap_time_rules import F1LapTimeRules

# Radio communication analysis rules
from utils.N04_nlp_rules import F1RadioRules  

# Gap analysis rules
from utils.N05_gap_rules import F1GapRules, calculate_all_gaps, calculate_gap_consistency, transform_gap_data_with_consistency

# Import standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import sys
import json
import time
import requests
# Import Experta components
from experta import Rule, NOT, OR, AND, AS, MATCH, TEST, EXISTS
from experta import DefFacts, Fact, Field, KnowledgeEngine

# Add parent directory to path to access modules
sys.path.append(os.path.abspath('../'))

# Configure visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_context("notebook", font_scale=1.2)

print("All components imported successfully!")

All components imported successfully!


---

## 2. Main Engine Class

This class inherits from all the enignes defined for each topic, and also a conflic resolution capability to avoid contradictory strategies being fired at the same time.

In [37]:
class F1CompleteStrategyEngine(F1DegradationRules, F1LapTimeRules, F1RadioRules, F1GapRules):
    """
    Unified strategy engine that integrates all rule systems:
    - Tire degradation rules 
    - Lap time prediction rules
    - Radio communication analysis rules
    - Gap analysis rules
    
    This class inherits from all specialized rule engines to combine their rules
    and adds conflict resolution capabilities.
    """
    
    def __init__(self):
        """Initialize the integrated engine"""
        # Call the parent constructor
        super().__init__()
        # Track which rule systems have fired rules
        self.active_systems = {
            'degradation': False,
            'lap_time': False,
            'radio': False,
            'gap': False
        }
    
    def get_recommendations(self):
        """
        Get all recommendations from the rule engine with enhanced conflict resolution.
        
        Returns:
            list: Sorted list of recommendations with conflicts resolved
        """
        # Get the base recommendations from parent method
        all_recommendations = super().get_recommendations()
        
        # If we have very few recommendations, no need for complex conflict resolution
        if len(all_recommendations) <= 2:
            return all_recommendations
        
        # Group recommendations by driver
        driver_recommendations = {}
        for rec in all_recommendations:
            driver = rec.get('DriverNumber', 0)  # Default to 0 if driver is not specified
            if driver not in driver_recommendations:
                driver_recommendations[driver] = []
            driver_recommendations[driver].append(rec)
        
        # Process each driver's recommendations for conflicts
        resolved_recommendations = []
        for driver, recs in driver_recommendations.items():
            # Only need conflict resolution if multiple recommendations
            if len(recs) > 1:
                resolved = self._resolve_conflicts(recs)
                resolved_recommendations.extend(resolved)
            else:
                # Single recommendation, no conflicts to resolve
                resolved_recommendations.extend(recs)
        
        # Sort by priority and confidence
        return sorted(
            resolved_recommendations,
            key=lambda x: (x.get('priority', 0), x.get('confidence', 0)),
            reverse=True
        )
    
    def _resolve_conflicts(self, recommendations):
        """
        Resolve conflicts between recommendations for the same driver.
        
        This method looks for contradictory recommendations and resolves them
        based on priority, confidence, and the nature of the conflict.
        
        Args:
            recommendations: List of recommendations for a single driver
            
        Returns:
            list: Resolved list of recommendations
        """
        # Group recommendations by action type
        action_groups = {}
        for rec in recommendations:
            action = rec['action']
            if action not in action_groups:
                action_groups[action] = []
            action_groups[action].append(rec)
        
        # Define conflicting action pairs
        conflicting_pairs = [
            # Can't extend stint and pit at the same time
            ('extend_stint', 'pit_stop'),
            ('extend_stint', 'prioritize_pit'),
            ('extend_stint', 'defensive_pit'),
            ('extend_stint', 'consider_pit'),
            
            # No need for preparation if immediate pit is recommended
            ('prepare_pit', 'pit_stop'),
            ('prepare_pit', 'prioritize_pit'),
            
            # Can't do undercut and overcut at the same time
            ('perform_undercut', 'perform_overcut')
        ]
        
        # Check for each conflict pair
        resolved = []
        excluded_recommendations = set()
        
        for action1, action2 in conflicting_pairs:
            if action1 in action_groups and action2 in action_groups:
                # We have a conflict!
                group1 = action_groups[action1]
                group2 = action_groups[action2]
                
                # Get the highest priority/confidence recommendation from each group
                best1 = max(group1, key=lambda x: (x.get('priority', 0), x.get('confidence', 0)))
                best2 = max(group2, key=lambda x: (x.get('priority', 0), x.get('confidence', 0)))
                
                # Compare and keep only the better one
                if (best1.get('priority', 0), best1.get('confidence', 0)) >= (best2.get('priority', 0), best2.get('confidence', 0)):
                    # best1 wins, exclude all from group2
                    excluded_recommendations.update(id(r) for r in group2)
                else:
                    # best2 wins, exclude all from group1
                    excluded_recommendations.update(id(r) for r in group1)
        
        # Add non-excluded recommendations
        for rec in recommendations:
            if id(rec) not in excluded_recommendations:
                resolved.append(rec)
        
        # Enhance the winning recommendations with context from conflicting ones
        if len(resolved) < len(recommendations):
            # We had conflicts and resolved them
            for rec in resolved:
                rec['explanation'] += " (Selected as optimal strategy after resolving conflicts)"
        
        return resolved
    
    def record_rule_fired(self, rule_name):
        """
        Record which rule fired and track which rule system it belongs to.
        
        Args:
            rule_name: Name of the rule that fired
        """
        # Standard recording from parent class
        super().record_rule_fired(rule_name)
        
        # Also track which system the rule belongs to
        if rule_name.startswith(('high_degradation', 'stint_extension', 'early_degradation')):
            self.active_systems['degradation'] = True
        elif rule_name.startswith(('optimal_performance', 'performance_cliff', 'post_traffic')):
            self.active_systems['lap_time'] = True
        elif rule_name.startswith(('grip_issue', 'weather_information', 'incident_reaction')):
            self.active_systems['radio'] = True
        elif rule_name.startswith(('undercut_opportunity', 'defensive_pit', 'strategic_overcut')):
            self.active_systems['gap'] = True

### 2.1 Testing class initialization

In [38]:
# Test that our integrated engine initializes correctly
try:
    engine = F1CompleteStrategyEngine()
    print("✅ Successfully created the integrated engine!")
    
    # Use reset to initialize the engine's working memory
    engine.reset()
    print("✅ Engine reset successful!")
    
    # Check MRO (Method Resolution Order) to confirm correct inheritance
    print("\nMethod Resolution Order:")
    for cls in F1CompleteStrategyEngine.__mro__:
        print(f"  - {cls.__name__}")
    
    # Print info about active systems
    print("\nActive rule systems:", engine.active_systems)
    
except Exception as e:
    print(f"❌ Error: {str(e)}")

✅ Successfully created the integrated engine!
✅ Engine reset successful!

Method Resolution Order:
  - F1CompleteStrategyEngine
  - F1DegradationRules
  - F1LapTimeRules
  - F1RadioRules
  - F1GapRules
  - F1StrategyEngine
  - KnowledgeEngine
  - object

Active rule systems: {'degradation': False, 'lap_time': False, 'radio': False, 'gap': False}


---

## 3. Transformation Function for integrating all the facts



In [39]:
# from utils.N01_agent_setup import calculate_race_phase

# def transform_all_facts(driver_number, tire_predictions=None, lap_predictions=None, 
#                        gap_data=None, radio_json_path=None, current_lap=None, 
#                        total_laps=66):
#     """
#     Transform all data sources into facts for the integrated rule engine.
    
#     This function combines fact transformation from all domains:
#     - Tire degradation
#     - Lap time prediction
#     - Gap analysis
#     - Radio communications
    
#     Args:
#         driver_number: Driver to focus on
#         tire_predictions: DataFrame with tire degradation predictions
#         lap_predictions: DataFrame with lap time predictions
#         gap_data: DataFrame with gap information
#         radio_json_path: Path to radio analysis JSON
#         current_lap: Current race lap
#         total_laps: Total race laps
        
#     Returns:
#         Dictionary of facts to declare in the engine
#     """
#     facts = {}
    
#     # 1. Transform tire degradation data
#     if tire_predictions is not None:
#         try:
#             tire_facts = transform_tire_predictions(tire_predictions, driver_number)
#             if tire_facts:
#                 # Ensure values are valid for the Fact schema
#                 if 'degradation' in tire_facts and tire_facts['degradation'] is not None:
#                     # For DegradationFact, ensure no None values in required fields
#                     deg_fact = tire_facts['degradation']
#                     for field_name in ['degradation_rate']:
#                         if field_name in deg_fact and deg_fact[field_name] is None:
#                             deg_fact[field_name] = 0.0  # Default to 0.0 for None values
                
#                 # For TelemetryFact, ensure no None values in required fields
#                 if 'telemetry' in tire_facts and tire_facts['telemetry'] is not None:
#                     telemetry_fact = tire_facts['telemetry']
#                     for field_name in ['tire_age', 'compound_id', 'driver_number', 'position']:
#                         if field_name in telemetry_fact and telemetry_fact[field_name] is None:
#                             if field_name in ['tire_age', 'compound_id', 'position']:
#                                 telemetry_fact[field_name] = 0  # Default to 0 for None values in int fields
#                             elif field_name == 'driver_number':
#                                 telemetry_fact[field_name] = driver_number  # Default to the driver number
                
#                 facts.update(tire_facts)
#                 print(f"✓ Transformed tire degradation data for Driver #{driver_number}")
#         except Exception as e:
#             print(f"✗ Error transforming tire data: {str(e)}")
    
#     # 2. Transform lap time data
#     if lap_predictions is not None:
#         try:
#             lap_facts = transform_lap_time_predictions(lap_predictions, driver_number)
#             if lap_facts:
#                 # Handle None values in telemetry fact to avoid schema validation errors
#                 if 'telemetry' in lap_facts and lap_facts['telemetry'] is not None:
#                     telemetry_fact = lap_facts['telemetry']
#                     # Ensure lap_time field has a valid float (not None)
#                     if 'lap_time' in telemetry_fact and telemetry_fact['lap_time'] is None:
#                         # Use predicted_lap_time as fallback, or 0.0 if that's None too
#                         if telemetry_fact.get('predicted_lap_time') is not None:
#                             telemetry_fact['lap_time'] = telemetry_fact['predicted_lap_time']
#                         else:
#                             telemetry_fact['lap_time'] = 0.0
                    
#                     # Ensure other required fields have valid values
#                     for field_name in ['predicted_lap_time', 'compound_id', 'tire_age', 'position']:
#                         if field_name in telemetry_fact and telemetry_fact[field_name] is None:
#                             # Default values for any None fields
#                             if field_name == 'predicted_lap_time':
#                                 telemetry_fact[field_name] = telemetry_fact.get('lap_time', 0.0)
#                             elif field_name in ['compound_id', 'tire_age', 'position']:
#                                 telemetry_fact[field_name] = 0
                
#                 facts.update(lap_facts)
#                 print(f"✓ Transformed lap time data for Driver #{driver_number}")
#         except Exception as e:
#             print(f"✗ Error transforming lap time data: {str(e)}")
    
#     # 3. Transform gap data
#     if gap_data is not None:
#         try:
#             # Ensure gap consistency is calculated
#             if 'consistent_gap_ahead_laps' not in gap_data.columns:
#                 gap_data = calculate_gap_consistency(gap_data)
#             gap_fact = transform_gap_data_with_consistency(gap_data, driver_number)
#             if gap_fact:
#                 # Handle None values in gap fact to avoid schema validation errors
#                 for field_name in ['gap_ahead', 'gap_behind', 'car_ahead', 'car_behind']:
#                     if field_name in gap_fact and gap_fact[field_name] is None:
#                         if field_name in ['gap_ahead', 'gap_behind']:
#                             gap_fact[field_name] = 0.0  # Default to 0.0 for distance gaps
#                         else:
#                             gap_fact[field_name] = 0  # Default to 0 for car numbers
                
#                 facts['gap'] = gap_fact
#                 print(f"✓ Transformed gap data for Driver #{driver_number}")
#         except Exception as e:
#             print(f"✗ Error transforming gap data: {str(e)}")
    
#     # 4. Transform radio data
#     if radio_json_path:
#         try:
#             radio_fact = transform_radio_analysis(radio_json_path)
#             if radio_fact:
#                 facts['radio'] = radio_fact
#                 print(f"✓ Transformed radio communication data")
#         except Exception as e:
#             print(f"✗ Error transforming radio data: {str(e)}")
    
#     # 5. Create race status fact (always required)
#     try:
#         from utils.N01_agent_setup import calculate_race_phase
#         race_phase = calculate_race_phase(current_lap, total_laps)
#         facts['race_status'] = RaceStatusFact(
#             lap=current_lap, 
#             total_laps=total_laps, 
#             race_phase=race_phase, 
#             track_status="clear"
#         )
#         print(f"✓ Created race status fact: Lap {current_lap}/{total_laps} ({race_phase})")
#     except Exception as e:
#         print(f"✗ Error creating race status fact: {str(e)}")
#         # Provide fallback race status
#         facts['race_status'] = RaceStatusFact(
#             lap=current_lap or 1,
#             total_laps=total_laps,
#             race_phase="mid",
#             track_status="clear"
#         )
    
#     return facts

In [40]:
from utils.N01_agent_setup import calculate_race_phase


def transform_all_facts(driver_number, tire_predictions=None, lap_predictions=None, 
                       gap_data=None, radio_json_path=None, current_lap=None, 
                       total_laps=66, debug=False):
    """
    Transform all data sources into facts for the integrated rule engine.
    
    This function combines fact transformation from all domains:
    - Tire degradation
    - Lap time prediction
    - Gap analysis
    - Radio communications
    
    Args:
        driver_number: Driver to focus on
        tire_predictions: DataFrame with tire degradation predictions
        lap_predictions: DataFrame with lap time predictions
        gap_data: DataFrame with gap information
        radio_json_path: Path to radio analysis JSON
        current_lap: Current race lap
        total_laps: Total race laps
        debug: If True, print detailed debug information
        
    Returns:
        Dictionary of facts to declare in the engine
    """
    facts = {}
    
    if debug:
        print(f"\n{'='*80}")
        print(f"TRANSFORMING FACTS FOR DRIVER #{driver_number} AT LAP {current_lap}")
        print(f"{'='*80}")
    
    # 1. Transform tire degradation data
    
    if tire_predictions is not None:
        try:
            tire_facts = transform_tire_predictions(tire_predictions, driver_number, current_lap)
            if tire_facts:
                # Ensure values are valid for the Fact schema
                if 'degradation' in tire_facts and tire_facts['degradation'] is not None:
                    deg_fact = tire_facts['degradation']
                    for field_name in ['degradation_rate']:
                        if field_name in deg_fact and deg_fact[field_name] is None:
                            deg_fact[field_name] = 0.0
                
                facts.update(tire_facts)
                print(f"✓ Transformed tire degradation data for Driver #{driver_number} at lap {current_lap}")
                if debug and 'degradation' in tire_facts:
                    print(f"  Degradation rate: {tire_facts['degradation'].get('degradation_rate')}")
                    print(f"  Tire age: {tire_facts['telemetry'].get('tire_age')}")
        except Exception as e:
            print(f"✗ Error transforming tire data: {str(e)}")
    
    # 2. Transform lap time data
    if lap_predictions is not None:
        try:
            lap_facts = transform_lap_time_predictions(lap_predictions, driver_number)
            if lap_facts:
                # Handle None values in telemetry fact to avoid schema validation errors
                if 'telemetry' in lap_facts and lap_facts['telemetry'] is not None:
                    telemetry_fact = lap_facts['telemetry']
                    # Ensure lap_time field has a valid float (not None)
                    if 'lap_time' in telemetry_fact and telemetry_fact['lap_time'] is None:
                        # Use predicted_lap_time as fallback, or 0.0 if that's None too
                        if telemetry_fact.get('predicted_lap_time') is not None:
                            telemetry_fact['lap_time'] = telemetry_fact['predicted_lap_time']
                        else:
                            telemetry_fact['lap_time'] = 0.0
                    
                    # Ensure other required fields have valid values
                    for field_name in ['predicted_lap_time', 'compound_id', 'tire_age', 'position']:
                        if field_name in telemetry_fact and telemetry_fact[field_name] is None:
                            # Default values for any None fields
                            if field_name == 'predicted_lap_time':
                                telemetry_fact[field_name] = telemetry_fact.get('lap_time', 0.0)
                            elif field_name in ['compound_id', 'tire_age', 'position']:
                                telemetry_fact[field_name] = 0
                
                facts.update(lap_facts)
                print(f"✓ Transformed lap time data for Driver #{driver_number}")
                if debug and 'telemetry' in lap_facts:
                    print(f"  Lap time: {lap_facts['telemetry'].get('lap_time')}")
                    print(f"  Predicted lap time: {lap_facts['telemetry'].get('predicted_lap_time')}")
        except Exception as e:
            print(f"✗ Error transforming lap time data: {str(e)}")
    
    # 3. Transform gap data
    if gap_data is not None:
        try:
            # Ensure gap consistency is calculated
            if 'consistent_gap_ahead_laps' not in gap_data.columns:
                print("  Calculating gap consistency...")
                gap_data = calculate_gap_consistency(gap_data)
            
            gap_fact = transform_gap_data_with_consistency(gap_data, driver_number)
            if gap_fact:
                # Handle None values in gap fact to avoid schema validation errors
                for field_name in ['gap_ahead', 'gap_behind', 'car_ahead', 'car_behind']:
                    if field_name in gap_fact and gap_fact[field_name] is None:
                        if field_name in ['gap_ahead', 'gap_behind']:
                            gap_fact[field_name] = 0.0  # Default to 0.0 for distance gaps
                        else:
                            gap_fact[field_name] = 0  # Default to 0 for car numbers
                
                # IMPORTANT: Fix - We need to convert the GapFact to a direct fact, not a nested dictionary
                facts['gap'] = gap_fact  # Store as-is for reference
                
                print(f"✓ Transformed gap data for Driver #{driver_number}")
                if debug:
                    print(f"  Gap ahead: {gap_fact.get('gap_ahead')}")
                    print(f"  Gap behind: {gap_fact.get('gap_behind')}")
                    print(f"  Consistent gap ahead laps: {gap_fact.get('consistent_gap_ahead_laps')}")
                    print(f"  Consistent gap behind laps: {gap_fact.get('consistent_gap_behind_laps')}")
                    print(f"  In undercut window: {gap_fact.get('in_undercut_window')}")
            else:
                print(f"✗ No gap data transformed for Driver #{driver_number}")
        except Exception as e:
            print(f"✗ Error transforming gap data: {str(e)}")
            import traceback
            if debug:
                traceback.print_exc()
    
    # 4. Transform radio data
    if radio_json_path:
        try:
            radio_fact = transform_radio_analysis(radio_json_path)
            if radio_fact:
                # IMPORTANT: Store the radio fact directly, not in a nested dictionary
                facts['radio'] = radio_fact
                
                print(f"✓ Transformed radio communication data")
                if debug:
                    print(f"  Sentiment: {radio_fact.get('sentiment')}")
                    print(f"  Intent: {radio_fact.get('intent')}")
                    print(f"  Entity categories: {list(radio_fact.get('entities', {}).keys())}")
            else:
                print(f"✗ No radio data transformed from {radio_json_path}")
        except Exception as e:
            print(f"✗ Error transforming radio data: {str(e)}")
            import traceback
            if debug:
                traceback.print_exc()
    
    # 5. Create race status fact (always required)
    try:
        race_phase = calculate_race_phase(current_lap, total_laps)
        facts['race_status'] = RaceStatusFact(
            lap=current_lap, 
            total_laps=total_laps, 
            race_phase=race_phase, 
            track_status="clear"
        )
        print(f"✓ Created race status fact: Lap {current_lap}/{total_laps} ({race_phase})")
    except Exception as e:
        print(f"✗ Error creating race status fact: {str(e)}")
        # Provide fallback race status
        facts['race_status'] = RaceStatusFact(
            lap=current_lap or 1,
            total_laps=total_laps,
            race_phase="mid",
            track_status="clear"
        )
    
    return facts

### 3.1 Function to test all rules have been correctly activated


In [41]:
def test_all_rule_activations():
    """
    Function to test that all types of rules activate correctly.
    
    This creates synthetic test cases for each rule system:
    - Tire degradation rules
    - Lap time rules
    - Gap rules 
    - Radio rules
    
    Returns:
        Dictionary mapping rule systems to activation status (True/False)
    """
    print(f"\n{'='*80}")
    print("TESTING ALL RULE ACTIVATIONS")
    print(f"{'='*80}")
    
    # Initialize the integrated engine
    engine = F1CompleteStrategyEngine()
    
    # Sample driver number
    driver_number = 44  # Lewis Hamilton
    
    # Test case 1: Radio rule activation (weather warning)
    print("\n=== TEST CASE 1: RADIO RULES (WEATHER WARNING) ===")
    weather_radio_message = "Warning: rain expected in 5 minutes, prepare wet tires"
    
    # Process the message
    print(f"Processing radio message: '{weather_radio_message}'")
    radio_json_path = process_radio_message(weather_radio_message)
    
    if radio_json_path:
        # Create a fresh engine instance
        engine.reset()
        
        # Transform the radio facts with debug mode
        facts = transform_all_facts(
            driver_number=driver_number,
            radio_json_path=radio_json_path,
            current_lap=20,
            total_laps=66,
            debug=True
        )
        
        # Declare all facts to the engine
        for key, fact in facts.items():
            try:
                engine.declare(fact)
                print(f"Declared {type(fact).__name__}")
            except Exception as e:
                print(f"Error declaring {key} fact: {e}")
        
        # Run the engine
        engine.run()
        
        # Check recommendations
        recommendations = engine.get_recommendations()
        print(f"\nGenerated {len(recommendations)} recommendations:")
        for i, rec in enumerate(recommendations):
            print(f"Recommendation {i+1}:")
            print(f"  Action: {rec['action']}")
            print(f"  Confidence: {rec['confidence']}")
            print(f"  Explanation: {rec['explanation']}")
    
    # Test case 2: Gap rule activation (undercut opportunity)
    print("\n=== TEST CASE 2: GAP RULES (UNDERCUT OPPORTUNITY) ===")
    
    # Create synthetic gap data
    import pandas as pd
    gap_data = pd.DataFrame({
        'DriverNumber': [driver_number],
        'GapToCarAhead': [1.5],        # Just within undercut window
        'GapToCarBehind': [3.0],
        'CarAheadNumber': [1],         # Max Verstappen
        'CarBehindNumber': [16],       # Charles Leclerc
        'Position': [2],
        'InUndercutWindow': [True],
        'InDRSWindow': [False],
        'LapNumber': [20]
    })
    
    # Calculate gap consistency manually (usually done by calculate_gap_consistency)
    gap_data['consistent_gap_ahead_laps'] = 5  # Enough for undercut rule
    gap_data['consistent_gap_behind_laps'] = 3
    gap_data['ahead_window'] = 'undercut_window'
    gap_data['behind_window'] = 'safe_window'
    
    # Create a fresh engine instance
    engine.reset()
    
    # Transform the gap facts with debug mode
    facts = transform_all_facts(
        driver_number=driver_number,
        gap_data=gap_data,
        current_lap=20,
        total_laps=66,
        debug=True
    )
    
    # Declare all facts to the engine
    for key, fact in facts.items():
        try:
            engine.declare(fact)
            print(f"Declared {type(fact).__name__}")
        except Exception as e:
            print(f"Error declaring {key} fact: {e}")
    
    # Run the engine
    engine.run()
    
    # Check recommendations
    recommendations = engine.get_recommendations()
    print(f"\nGenerated {len(recommendations)} recommendations:")
    for i, rec in enumerate(recommendations):
        print(f"Recommendation {i+1}:")
        print(f"  Action: {rec['action']}")
        print(f"  Confidence: {rec['confidence']}")
        print(f"  Explanation: {rec['explanation']}")
    
    # Test case 3: Lap Time rule activation (performance cliff detection)
    print("\n=== TEST CASE 3: LAP TIME RULES (PERFORMANCE CLIFF) ===")
    
    # Create synthetic lap time data
    lap_predictions = pd.DataFrame({
        'DriverNumber': [driver_number],
        'LapNumber': [25],
        'LapTime': [85.0],                 # Current lap time
        'PredictedLapTime': [86.5],        # Predicted to be slower by 1.5s (performance cliff)
        'CompoundID': [2],                 # Medium tire
        'TyreAge': [18],                   # Older tires (>15 laps)
        'Position': [3],
        'IsNextLapPrediction': [True]
    })
    
    # Create a fresh engine instance
    engine.reset()
    
    # Transform lap time data with debug mode
    facts = transform_all_facts(
        driver_number=driver_number,
        lap_predictions=lap_predictions,
        current_lap=25,
        total_laps=66,
        debug=True
    )
    
    # Declare all facts to the engine
    for key, fact in facts.items():
        try:
            engine.declare(fact)
            print(f"Declared {type(fact).__name__}")
        except Exception as e:
            print(f"Error declaring {key} fact: {e}")
    
    # Run the engine
    engine.run()
    
    # Check recommendations
    recommendations = engine.get_recommendations()
    print(f"\nGenerated {len(recommendations)} recommendations:")
    for i, rec in enumerate(recommendations):
        print(f"Recommendation {i+1}:")
        print(f"  Action: {rec['action']}")
        print(f"  Confidence: {rec['confidence']}")
        print(f"  Explanation: {rec['explanation']}")
    
    # Test case 4: Degradation rule activation (high degradation)
    print("\n=== TEST CASE 4: DEGRADATION RULES (HIGH DEGRADATION) ===")
    
    # Create synthetic tire prediction data
    # Test case 4: Degradation rule activation (high degradation)
    print("\n=== TEST CASE 4: DEGRADATION RULES (HIGH DEGRADATION) ===")

    # Create synthetic tire prediction data with proper length arrays
    tire_predictions = pd.DataFrame({
        'DriverNumber': [driver_number, driver_number, driver_number],
        'Stint': [1, 1, 1],
        'CurrentTyreAge': [15, 15, 15],             # Same age for all predictions
        'LapsAheadPred': [1, 2, 3],                 # Predictions for next 3 laps
        'PredictedDegradationRate': [0.35, 0.38, 0.42],  # Very high degradation (>0.3)
        'CompoundID': [1, 1, 1],                    # Soft tire
        'Position': [4, 4, 4], 
        'FuelAdjustedDegPercent': [12.0, 12.5, 13.0]
    })

    # Create a fresh engine instance
    engine.reset()

    # Transform tire data with debug mode
    facts = transform_all_facts(
        driver_number=driver_number,
        tire_predictions=tire_predictions,
        current_lap=30,
        total_laps=66,
        debug=True
    )
    
    # Declare all facts to the engine
    for key, fact in facts.items():
        try:
            engine.declare(fact)
            print(f"Declared {type(fact).__name__}")
        except Exception as e:
            print(f"Error declaring {key} fact: {e}")
    
    # Run the engine
    engine.run()
    
    # Check recommendations
    recommendations = engine.get_recommendations()
    print(f"\nGenerated {len(recommendations)} recommendations:")
    for i, rec in enumerate(recommendations):
        print(f"Recommendation {i+1}:")
        print(f"  Action: {rec['action']}")
        print(f"  Confidence: {rec['confidence']}")
        print(f"  Explanation: {rec['explanation']}")
    
    # Check final activation status
    activity_status = {
        'radio_rules': bool(engine.active_systems.get('radio', False)),
        'gap_rules': bool(engine.active_systems.get('gap', False)),
        'lap_time_rules': bool(engine.active_systems.get('lap_time', False)),
        'degradation_rules': bool(engine.active_systems.get('degradation', False))
    }
    
    print("\n=== FINAL RULE ACTIVATION STATUS ===")
    for rule_system, is_active in activity_status.items():
        status = "✓ ACTIVE" if is_active else "✗ INACTIVE"
        print(f"{status}: {rule_system}")
    
    return activity_status

In [42]:
# test_all_rule_activations()

---

## 4. Load and prepare main function


In [43]:
def load_all_data(race_data_path, models_path=None, lap_model_path=None, gap_data_path=None, radio_message=None):
    """
    Load all necessary data for the strategy engine from various sources.
    
    Args:
        race_data_path: Path to race telemetry CSV
        models_path: Path to tire prediction models directory
        lap_model_path: Path to lap time prediction model file
        gap_data_path: Optional path to gap data CSV
        radio_message: Optional radio message text to analyze
        
    Returns:
        Dictionary with all loaded data
    """
    result = {}
    
    # 1. Load race telemetry data (required)
    print("Loading race telemetry data...")
    try:
        race_data = pd.read_csv(race_data_path)
        result['race_data'] = race_data
        print(f"✓ Loaded race data: {len(race_data)} rows")
    except Exception as e:
        print(f"✗ Could not load race data: {str(e)}")
        return result  # Cannot continue without race data
    
    # 2. Generate tire degradation predictions
    if models_path:
        print("Generating tire degradation predictions...")
        try:
            # Default monitoring thresholds by compound
            compound_thresholds = {1: 6, 2: 12, 3: 25}
            tire_predictions = load_tire_predictions(
                race_data, 
                models_path, 
                compound_thresholds=compound_thresholds
            )
            result['tire_predictions'] = tire_predictions
            print(f"✓ Generated tire predictions: {len(tire_predictions) if tire_predictions is not None else 0} rows")
        except Exception as e:
            print(f"✗ Could not generate tire predictions: {str(e)}")
    
    # 3. Generate lap time predictions with a separate model path
    if lap_model_path:
        print("Generating lap time predictions...")
        try:
            # Import the lap prediction module directly
            sys.path.append(os.path.abspath('../'))
            from ML_tyre_pred.ML_utils.N00_model_lap_prediction import predict_lap_times
            
            # Use the function directly instead of the wrapper
            lap_predictions = predict_lap_times(
                race_data,
                model_path=lap_model_path,
                include_next_lap=True
            )
            
            result['lap_predictions'] = lap_predictions
            print(f"✓ Generated lap time predictions: {len(lap_predictions) if lap_predictions is not None else 0} rows")
        except Exception as e:
            print(f"✗ Could not generate lap time predictions: {str(e)}")
    
    # 4. Create gap data from race data if not provided as CSV
    if gap_data_path:
        print(f"Loading gap data from {gap_data_path}...")
        try:
            gap_data = pd.read_csv(gap_data_path)
            result['gap_data'] = gap_data
            print(f"✓ Loaded gap data: {len(gap_data)} rows")
        except Exception as e:
            print(f"✗ Could not load gap data: {str(e)}")
    
    # 5. Process radio message if provided
    if radio_message:
        print(f"Processing radio message: '{radio_message}'...")
        try:
            json_path = process_radio_message(radio_message)
            if json_path:
                result['radio_json_path'] = json_path
                print(f"✓ Processed radio message to: {json_path}")
        except Exception as e:
            print(f"✗ Could not process radio message: {str(e)}")
    
    return result

## 5. Complete end-to-end pipeline

### 5.1 Analyze the strategy with no radio (deprecated)

In [44]:
def analyze_strategy(
    driver_number, 
    race_data_path, 
    models_path=None,
    lap_model_path=None,
    gap_data_path=None,
    radio_message=None, 
    current_lap=None, 
    total_laps=66
):
    """
    Complete end-to-end F1 strategy analysis pipeline.
    
    This function integrates all components of the F1 strategy system:
    1. Loads and prepares data from all sources
    2. Transforms data into facts for the rule engine
    3. Runs the integrated engine with all rule systems
    4. Returns prioritized strategy recommendations
    
    Args:
        driver_number: Driver number to analyze
        race_data_path: Path to race telemetry CSV
        models_path: Path to tire prediction models
        lap_model_path: Path to lap time prediction model
        gap_data_path: Optional path to gap data CSV
        radio_message: Optional radio message to analyze
        current_lap: Current lap number (defaults to max in data)
        total_laps: Total race laps
        
    Returns:
        List of strategy recommendations
    """
    start_time = time.time()
    print(f"\n{'='*80}")
    print(f"F1 STRATEGY ANALYSIS FOR DRIVER #{driver_number}")
    print(f"{'='*80}")
    
    # Step 1: Load all data sources
    print("\n--- LOADING DATA ---")
    data = load_all_data(race_data_path, models_path, lap_model_path, gap_data_path, radio_message)
    
    # Determine current lap if not provided
    if current_lap is None and 'race_data' in data:
        try:
            driver_data = data['race_data'][data['race_data']['DriverNumber'] == driver_number]
            if not driver_data.empty:
                if 'LapNumber' in driver_data.columns:
                    current_lap = int(driver_data['LapNumber'].max())
                else:
                    # Use TyreAge as fallback
                    current_lap = int(driver_data['TyreAge'].max())
                print(f"✓ Determined current lap: {current_lap}")
            else:
                current_lap = 20  # Default mid-race
                print(f"✓ Using default lap: {current_lap}")
        except:
            current_lap = 20  # Default
            print(f"✓ Using default lap: {current_lap}")
    
    # Step 2: Transform data into facts
    print("\n--- TRANSFORMING DATA INTO FACTS ---")
    facts = transform_all_facts(
        driver_number=driver_number,
        tire_predictions=data.get('tire_predictions'),
        lap_predictions=data.get('lap_predictions'),
        gap_data=data.get('gap_data'),
        radio_json_path=data.get('radio_json_path'),
        current_lap=current_lap,
        total_laps=total_laps
    )
    
    # Step 3: Run the integrated strategy engine
    print("\n--- RUNNING INTEGRATED STRATEGY ENGINE ---")
    engine = F1CompleteStrategyEngine()
    engine.reset()
    
    # Declare facts to the engine
    for fact_type, fact in facts.items():
        if fact is not None:
            try:
                engine.declare(fact)
                print(f"✓ Declared {type(fact).__name__}")
            except Exception as e:
                print(f"✗ Error declaring {type(fact).__name__}: {str(e)}")
    
    # Run the engine - this will activate all applicable rules
    print("\nExecuting rules...")
    engine.run()
    
    # Get recommendations with conflict resolution
    recommendations = engine.get_recommendations()
    
    # Step 4: Show results
    print(f"\n--- ANALYSIS RESULTS ---")
    print(f"Generated {len(recommendations)} strategy recommendations")
    
    if recommendations:
        for i, rec in enumerate(recommendations):
            print(f"\nRecommendation {i+1}:")
            print(f"  Action: {rec['action']}")
            print(f"  Confidence: {rec['confidence']:.2f}")
            print(f"  Priority: {rec['priority']}")
            print(f"  Explanation: {rec['explanation']}")
    else:
        print("No recommendations generated. Try with different data inputs.")
    
    # Show which rule systems were activated
    print("\nActivated rule systems:")
    for system, active in engine.active_systems.items():
        status = "✓" if active else "✗"
        print(f"  {status} {system.capitalize()} rules")
    
    elapsed_time = time.time() - start_time
    print(f"\nAnalysis completed in {elapsed_time:.2f} seconds")
    
    return recommendations

---

### 5.2 Extracting the 2023 Spanish GP radios

In [45]:
def extract_spain_gp_radios(output_directory):
    """
    Extract team radio messages from the 2023 Spanish Grand Prix using OpenF1 API.
    
    Args:
        output_directory: Folder where MP3 files will be saved
        
    Returns:
        DataFrame with radio metadata and paths to MP3 files
    """
    
    
    print(f"\n{'='*80}")
    print(f"EXTRACTING TEAM RADIOS FROM 2023 SPANISH GP")
    print(f"{'='*80}")
    
    # Create output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)
    
    # Spanish GP data
    spain_gp = {"name": "Spain", "session_key": 9102, "year": 2023}
    
    # Extract team radio data from OpenF1 API
    url = f"https://api.openf1.org/v1/team_radio?session_key={spain_gp['session_key']}"
    print(f"Fetching data from: {url}")
    
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    print(f"✓ Found {len(data)} radio messages for Spanish GP")
    
    # Convert to DataFrame and process dates
    df = pd.DataFrame(data)
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
    
    # Add GP name column for identification
    df['gp_name'] = spain_gp['name']
    
    # Add column to store MP3 file path
    df['mp3_path'] = None
    
    # Save metadata as CSV for reference
    metadata_path = os.path.join(output_directory, "spain_2023_radio_metadata.csv")
    df.to_csv(metadata_path, index=False)
    print(f"Metadata saved to: {metadata_path}")
    
    # Download audio files
    print("\nDownloading audio files...")
    
    # Group by driver
    grouped = df.groupby(["driver_number"])
    
    total_downloads = 0
    for driver_number, group in grouped:
        # Create folder for each driver
        driver_folder = os.path.join(output_directory, str(driver_number))
        os.makedirs(driver_folder, exist_ok=True)
        
        # Download and save audio files
        for i, row in group.iterrows():
            url = row["recording_url"]
            if pd.isna(url):
                continue
            
            # Create filename including index for sorting
            filename = f"radio_{i}.mp3"
            output_path = os.path.join(driver_folder, filename)
            
            # Save path in DataFrame
            df.loc[i, 'mp3_path'] = output_path
            
            # Check if file already exists to avoid duplicate downloads
            if os.path.exists(output_path):
                continue
            
            # Download the file
            try:
                response = requests.get(url)
                response.raise_for_status()
                with open(output_path, "wb") as f:
                    f.write(response.content)
                print(f"Downloaded: {output_path}")
                total_downloads += 1
                
                # Small delay to avoid overloading the server
                time.sleep(0.5)
            except Exception as e:
                print(f"Error downloading {url}: {e}")
    
    # Save updated metadata with mp3_path values
    df.to_csv(metadata_path, index=False)
    print(f"Updated metadata saved to: {metadata_path}")
    
    print(f"Total files downloaded: {total_downloads}")
    
    # Display statistics of communications per driver
    print("\nDistribution of communications per driver:")
    driver_counts = df['driver_number'].value_counts().sort_index()
    for driver_num, count in driver_counts.items():
        print(f"  • Driver #{driver_num}: {count} communications")
    
    return df

---

### Processing the radios with NLP models 

In [46]:
def process_spain_gp_radios(radio_metadata_df, processed_directory):
    """
    Process radio MP3 files by transcribing and analyzing them with NLP.
    Includes driver number in the JSON filenames for better organization.
    
    Args:
        radio_metadata_df: DataFrame with radio metadata
        processed_directory: Folder to save processed results
    
    Returns:
        Dict: Mapping of {(driver_number, index): json_path}
    """
    import os
    import time
    import shutil
    from datetime import timedelta
    
    print(f"\n{'='*80}")
    print(f"PROCESSING TEAM RADIO FILES FROM SPANISH GP")
    print(f"{'='*80}")
    
    # Ensure output directory exists
    os.makedirs(processed_directory, exist_ok=True)
    
    # Dictionary to store {(driver_number, index): json_path}
    processed_radios = {}
    
    # Filter rows with valid MP3 files
    valid_radios = radio_metadata_df.dropna(subset=['mp3_path'])
    print(f"Processing {len(valid_radios)} radio messages...")
    
    for i, row in valid_radios.iterrows():
        driver_number = row['driver_number']
        mp3_path = row['mp3_path']
        
        # Verify file exists
        if not os.path.exists(mp3_path):
            print(f"File not found: {mp3_path}")
            continue
        
        print(f"Processing radio {i+1}/{len(valid_radios)}: Driver #{driver_number}")
        
        try:
            # Use our existing function to transcribe and analyze
            json_path = process_radio_message(mp3_path, is_audio=True)
            
            if json_path:
                # Get the original filename and create a new one with driver number
                original_filename = os.path.basename(json_path)
                
                # Extract timestamp from original filename
                filename_parts = original_filename.split('_')
                if len(filename_parts) >= 3 and filename_parts[0] == 'radio' and filename_parts[1] == 'analysis':
                    # Extract date and time portions
                    date_part = filename_parts[2]  # e.g. "20250419"
                    time_part = filename_parts[3].split('.')[0] if len(filename_parts) > 3 else "000000"
                    
                    # Create new filename with driver number AND timestamp preserved
                    new_filename = f"driver_{driver_number}_radio_{date_part}_{time_part}.json"
                else:
                    # Fallback for unexpected filename formats
                    new_filename = f"driver_{driver_number}_{original_filename}"
                
                new_json_path = os.path.join(processed_directory, new_filename)
                
                # Copy the file
                shutil.copy2(json_path, new_json_path)
                print(f"  ✓ Copied and renamed to: {new_json_path}")
                
                # Store the association with the NEW path
                processed_radios[(driver_number, i)] = new_json_path
            else:
                print(f"  ✗ Error processing audio, no JSON generated")
        
        except Exception as e:
            print(f"  ✗ Error processing {mp3_path}: {str(e)}")
        
        # Brief pause to avoid issues
        time.sleep(0.1)
    
    print(f"\nTotal radios processed: {len(processed_radios)}")
    return processed_radios

---

### 5.4 Mapping radios to laps


In [47]:
def map_radios_to_laps(radio_metadata_df, race_data_df):
    """
    Associate each radio message with the corresponding race lap.
    Uses timing information to estimate which lap each message was sent.
    
    Args:
        radio_metadata_df: DataFrame with radio metadata
        race_data_df: DataFrame with race data
    
    Returns:
        Dict: Mapping of {(driver_number, lap): message_data}
    """
    
    print(f"\n{'='*80}")
    print(f"MAPPING RADIO MESSAGES TO RACE LAPS")
    print(f"{'='*80}")
    
    # Prepare mapping dictionary {(driver_number, lap): message_data}
    radio_lap_mapping = {}
    
    # Verify data has necessary columns
    if 'date' not in radio_metadata_df.columns:
        print("Error: Radio metadata doesn't contain 'date' column")
        return radio_lap_mapping
    
    # Get race start and end times
    if 'date' in race_data_df.columns:
        race_start = race_data_df['date'].min()
        race_end = race_data_df['date'].max()
    else:
        print("No date column in race data. Using radio timestamps")
        race_start = radio_metadata_df['date'].min()
        race_end = radio_metadata_df['date'].max()
    
    # Calculate total race duration
    race_duration = (race_end - race_start).total_seconds()
    
    # Get total number of laps
    # Try different columns (LapNumber, TyreAge) or default to 66 (typical Spanish GP)
    total_laps = int(race_data_df['LapNumber'].max() if 'LapNumber' in race_data_df.columns else 
                     race_data_df['TyreAge'].max() if 'TyreAge' in race_data_df.columns else 66)
    
    print(f"Race duration: {race_duration/60:.1f} minutes, Total laps: {total_laps}")
    
    # For each radio, estimate lap based on timing
    for i, row in radio_metadata_df.iterrows():
        if pd.isna(row['date']):
            continue
        
        driver_number = row['driver_number']
        radio_time = row['date']
        
        # Calculate elapsed time since race start (in seconds)
        elapsed_time = (radio_time - race_start).total_seconds()
        
        # Estimate lap based on elapsed time
        # Assumes uniform time distribution across laps
        estimated_lap = max(1, min(total_laps, int(elapsed_time / race_duration * total_laps)))
        
        # Save the association with data to identify the message
        radio_lap_mapping[(driver_number, estimated_lap)] = {
            'radio_index': i,
            'mp3_path': row.get('mp3_path'),
            'transcript_path': None,  # Will be filled later with process_radio_message
            'recording_url': row.get('recording_url'),
            'radio_time': radio_time
        }
    
    print(f"Mapped {len(radio_lap_mapping)} radios to specific laps")
    
    # Calculate distribution of radios per lap
    laps_distribution = {}
    for (_, lap), _ in radio_lap_mapping.items():
        laps_distribution[lap] = laps_distribution.get(lap, 0) + 1
    
    print("\nDistribution of radios by lap (first 10 laps):")
    for lap in sorted(laps_distribution.keys())[:10]:
        print(f"  • Lap {lap}: {laps_distribution[lap]} radios")
    
    return radio_lap_mapping

---

### 5.5 Running example analysis (deprecated)

In [48]:
def run_example_analysis():
    """
    Example demonstrating the complete strategy analysis pipeline.
    
    Uses the provided CSV files and a sample radio message to run
    a full strategy analysis for a specific driver.
    """
    print("\n=== RUNNING EXAMPLE STRATEGY ANALYSIS ===")
    
    # Define paths to data files
    race_data_path = '../../outputs/week3/lap_prediction_data.csv'
    models_path = '../../outputs/week5/models/'
    lap_model_path = '../../outputs/week3/xgb_sequential_model.pkl'  # Separate path for lap model
    
    # Sample radio message
    radio_message = "Box this lap for softs, there's rain expected in 10 minutes"
    
    # Driver to analyze (Lewis Hamilton)
    driver_number = 44
    
    # Run the analysis
    recommendations = analyze_strategy(
        driver_number=driver_number,
        race_data_path=race_data_path,
        models_path=models_path,
        lap_model_path=lap_model_path,
        radio_message=radio_message,
        current_lap=20,  # Mid-race scenario
        total_laps=66    # Typical F1 race length
    )
    
    return recommendations



In [49]:
# example_recommendations = run_example_analysis()

---

## 6. Function for merging all the models.

In [50]:
def analyze_all_drivers_with_real_radios(
    race_data_path, 
    models_path=None,
    lap_model_path=None,
    output_path=None,
    extract_new_radios=True
):
    """
    Analyze strategy for all drivers at key race moments using real radio 
    communications from the 2023 Spanish GP, with proper gap data processing using
    the approach from analyze_race_gaps.
    
    Args:
        race_data_path: Path to race telemetry CSV
        models_path: Path to tire degradation models
        lap_model_path: Path to lap time prediction model
        output_path: Path to save results
        extract_new_radios: If True, extract new radio data; if False, use existing data
    
    Returns:
        DataFrame with all recommendations
    """
    import glob
    from datetime import datetime
    import fastf1
    import re
    
    print(f"\n{'='*80}")
    print(f"ANALYZING ALL DRIVERS WITH REAL RADIOS FROM 2023 SPANISH GP")
    print(f"{'='*80}")
    print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # 1. Set up folder paths
    base_dir = '../../outputs'
    radio_dir = os.path.join(base_dir, "week6", "radios")
    processed_dir = os.path.join(base_dir, "week6", "processed_radios")
    
    # Create necessary directories
    os.makedirs(radio_dir, exist_ok=True)
    os.makedirs(processed_dir, exist_ok=True)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # 2. Load race data for tire and lap time predictions
    race_data = pd.read_csv(race_data_path)
    print(f"Race data loaded: {len(race_data)} rows")
    
    # 3. Extract radios from OpenF1 if needed
    radio_metadata_path = os.path.join(radio_dir, "spain_2023_radio_metadata.csv")
    
    if extract_new_radios or not os.path.exists(radio_metadata_path):
        print("Extracting new radio data...")
        radio_metadata = extract_spain_gp_radios(radio_dir)
    else:
        print(f"Loading existing radio metadata: {radio_metadata_path}")
        radio_metadata = pd.read_csv(radio_metadata_path)
        if 'date' in radio_metadata.columns:
            radio_metadata['date'] = pd.to_datetime(radio_metadata['date'], errors='coerce')
    
    # 4. Map radios to laps
    radio_lap_mapping = map_radios_to_laps(radio_metadata, race_data)
    
    # 5. Process radios - Scan for individual files
    # Scan the processed_radios directory for all JSON files
    json_pattern = os.path.join(processed_dir, "*.json")
    json_files = glob.glob(json_pattern)
    
    print(f"Found {len(json_files)} processed radio JSON files")
    
    # Create a mapping from driver number to JSON files
    driver_json_mapping = {}
    for json_file in json_files:
        # Extract driver number from filename (driver_X_radio_*.json)
        file_name = os.path.basename(json_file)
        match = re.match(r'driver_(\d+)_radio_.*\.json', file_name)
        if match:
            driver_number = int(match.group(1))
            if driver_number not in driver_json_mapping:
                driver_json_mapping[driver_number] = []
            driver_json_mapping[driver_number].append(json_file)
    
    print(f"Created mapping for {len(driver_json_mapping)} drivers with radio data")
    
    # 6. Generate tire and lap time predictions
    tire_predictions = None
    if models_path:
        try:
            compound_thresholds = {1: 6, 2: 12, 3: 25}
            tire_predictions = load_tire_predictions(race_data, models_path, compound_thresholds)
            print(f"Tire predictions generated: {len(tire_predictions) if tire_predictions is not None else 0} rows")
        except Exception as e:
            print(f"Error generating tire predictions: {str(e)}")
    
    lap_predictions = None
    if lap_model_path:
        try:
            lap_predictions = load_lap_time_predictions(race_data, lap_model_path)
            print(f"Lap time predictions generated: {len(lap_predictions) if lap_predictions is not None else 0} rows")
        except Exception as e:
            print(f"Error generating lap time predictions: {str(e)}")
    
    # 7. Load gap data using FastF1
    try:
        # Setup FastF1 cache
        fastf1.Cache.enable_cache('../../f1-strategy/f1_cache')
        
        # Spanish GP 2023 details
        year = 2023
        grand_prix = 'Spain'
        session_type = 'R'  # R for Race
        
        gap_data = None
        gap_data_path = os.path.join(base_dir, "week6", "gap_data.csv")
        
        # Try to load cached gap data first
        if os.path.exists(gap_data_path):
            try:
                print(f"Loading gap data from: {gap_data_path}")
                gap_data = pd.read_csv(gap_data_path)
                print(f"Gap data loaded: {len(gap_data)} rows")
            except Exception as e:
                print(f"Error loading gap data: {str(e)}")
                gap_data = None
        
        # If no cached data or loading failed, calculate using FastF1
        if gap_data is None:
            print("\nLoading FastF1 session data for gap calculation...")
            race = fastf1.get_session(year, grand_prix, session_type)
            race.load()
            laps_data = race.laps
            print(f"FastF1 data loaded with {len(laps_data)} lap entries")
            
            print("\nCalculating gaps between cars...")
            gap_data = calculate_all_gaps(laps_data)
            
            # Save gap data for future use
            gap_data.to_csv(gap_data_path, index=False)
            print(f"Gap data calculated and saved: {len(gap_data)} rows")
        
        # Ensure gap data has consistency metrics
        if gap_data is not None and 'consistent_gap_ahead_laps' not in gap_data.columns:
            print("\nCalculating gap consistency metrics...")
            gap_data = calculate_gap_consistency(gap_data)
            # Update saved file with consistency data
            gap_data.to_csv(gap_data_path, index=False)
            print("Gap consistency metrics added")
    
    except Exception as e:
        print(f"Error working with FastF1 data: {str(e)}")
        import traceback
        traceback.print_exc()
        gap_data = None
    
    # 8. Define total laps and strategic points
    # Spanish GP 2023 had 66 laps
    total_laps = 66
    
    # Verify from gap data if available
    if gap_data is not None and 'LapNumber' in gap_data.columns:
        total_laps_from_data = int(gap_data['LapNumber'].max())
        if total_laps_from_data > 0:
            total_laps = total_laps_from_data
            print(f"Confirmed total laps from gap data: {total_laps}")
    
    # Define strategic points for analysis using compound thresholds and intervals
    compound_thresholds = {1: 6, 2: 12, 3: 25}  # Soft: 6, Medium: 12, Hard: 25
    interval = 5  # Interval between recommendations (puedes ajustar este valor)
    strategic_points = set()
    for start_lap in compound_thresholds.values():
        strategic_points.update(range(start_lap, total_laps + 1, interval))
    strategic_points = sorted(strategic_points)
    print(f"Strategic analysis points: Laps {strategic_points}")
    
    # 9. Process each driver
    all_recommendations = []
    
    # Define compound names mapping for readable compound names
    compound_names = {
        1: 'Soft',
        2: 'Medium',
        3: 'Hard',
        0: 'Unknown'
    }
    
    # Use driver numbers from race data (convert to standard Python int)
    drivers = [int(driver) for driver in race_data['DriverNumber'].unique()]

    # Add any drivers from gap data that might not be in race data
    if gap_data is not None and 'DriverNumber' in gap_data.columns:
        gap_drivers = gap_data['DriverNumber'].unique()
        # Add any drivers from gap data that aren't already in our list
        for driver in gap_drivers:
            driver_int = int(driver)  # Convert to standard integer
            if driver_int not in drivers:
                drivers.append(driver_int)
        
    # Now sort the list of integers
    drivers = sorted(drivers)
    
    print(f"\n--- ANALYZING {len(drivers)} DRIVERS ---")
    
    # 10. Process each driver at each strategic point
    for driver_number in drivers:
        print(f"\nAnalyzing driver #{driver_number}")
        
        # Process each strategic point
        for lap in strategic_points:
            print(f"  Lap {lap}/{total_laps}")
            
            # Find a radio message for this driver at or near this lap
            radio_json_path = None
            
            # First check if we have any radio files for this driver
            if driver_number in driver_json_mapping and driver_json_mapping[driver_number]:
                # Just use the first radio file for now (simplification)
                radio_json_path = driver_json_mapping[driver_number][0]
                print(f"  Using radio file: {os.path.basename(radio_json_path)}")
            
            # Filter gap data for this driver and lap
            driver_gap_data = None
            if gap_data is not None:
                # Filter for driver and relevant lap
                driver_data = gap_data[gap_data['DriverNumber'] == driver_number]
                if not driver_data.empty:
                    # Find gap data for this lap or closest previous lap
                    relevant_laps = driver_data[driver_data['LapNumber'] <= lap]
                    if not relevant_laps.empty:
                        closest_lap = relevant_laps['LapNumber'].max()
                        driver_gap_data = relevant_laps[relevant_laps['LapNumber'] == closest_lap]
                        print(f"  Using gap data from lap {closest_lap}")
            
            # Transform tire predictions for this driver
            driver_tire_predictions = None
            if tire_predictions is not None:
                driver_tire_data = tire_predictions[tire_predictions['DriverNumber'] == driver_number]
                if not driver_tire_data.empty:
                    # ✅ NUEVO: Filtrar por vuelta actual si existe una columna adecuada
                    if 'RaceLap' in driver_tire_data.columns:
                        # Filtrar datos que sean relevantes para esta vuelta
                        relevant_data = driver_tire_data[driver_tire_data['RaceLap'] <= lap]
                        if not relevant_data.empty:
                            driver_tire_predictions = relevant_data
                        else:
                            driver_tire_predictions = driver_tire_data
                    else:
                        driver_tire_predictions = driver_tire_data
                    print(f"  Using tire predictions ({len(driver_tire_predictions)} rows) for lap {lap}")
            
            # Transform lap time predictions for this driver
            driver_lap_predictions = None
            if lap_predictions is not None:
                driver_lap_data = lap_predictions[lap_predictions['DriverNumber'] == driver_number]
                if not driver_lap_data.empty:
                    driver_lap_predictions = driver_lap_data
                    print(f"  Using lap time predictions ({len(driver_lap_data)} rows)")
            
            # Use transform_all_facts to prepare facts
            facts = transform_all_facts(
                driver_number=driver_number,
                tire_predictions=driver_tire_predictions,
                lap_predictions=driver_lap_predictions,
                gap_data=driver_gap_data,
                radio_json_path=radio_json_path,
                current_lap=lap,  # ✅ CRITICAL: Passing the current lap
                total_laps=total_laps,
                debug=True  # ✅ Enable debug to see what's happening
            )
            
            # Create rule engine
            engine = F1CompleteStrategyEngine()
            engine.reset()
            
            # Declare all facts to the engine
            for key, fact in facts.items():
                if fact is not None:
                    try:
                        engine.declare(fact)
                        print(f"  Declared {type(fact).__name__}")
                    except Exception as e:
                        print(f"  Error declaring {key} fact: {str(e)}")
            
            # Run the engine
            engine.run()
            
            # Get recommendations
            recommendations = engine.get_recommendations()
            
            # Create a mapping from action to originating rule
            action_to_rule = {
                # Tire Degradation Rules
                "pit_stop": "high_degradation_pit_stop",  # Note: also from performance_cliff_detection
                "extend_stint": "stint_extension_recommendation",
                "prepare_pit": "early_degradation_warning",
                "consider_pit": "predicted_high_degradation_alert",
                
                # Lap Time Rules
                "push_strategy": "optimal_performance_window",
                # "pit_stop" is duplicated from above
                "recovery_push": "post_traffic_recovery",
                
                # NLP Radio Rules
                "prioritize_pit": "grip_issue_response",
                "prepare_rain_tires": "weather_information_adjustment",
                "reevaluate_pit_window": "incident_reaction",
                
                # Gap Analysis Rules
                "perform_undercut": "undercut_opportunity",
                "defensive_pit": "defensive_pit_stop",
                "perform_overcut": "strategic_overcut",
                "adjust_pit_window": "traffic_management"
            }
            
            # Handle ambiguous action "pit_stop" which can come from multiple rules
            # Look at active_systems to determine which system is active
            if len(recommendations) > 0 and "pit_stop" in [r["action"] for r in recommendations]:
                if engine.active_systems.get('degradation', False):
                    # If degradation system is active, pit_stop is from high_degradation_pit_stop
                    for rec in recommendations:
                        if rec["action"] == "pit_stop":
                            rec["rule_fired"] = "high_degradation_pit_stop"
                elif engine.active_systems.get('lap_time', False):
                    # If lap_time system is active, pit_stop is from performance_cliff_detection
                    for rec in recommendations:
                        if rec["action"] == "pit_stop":
                            rec["rule_fired"] = "performance_cliff_detection"
            
            # Add rule_fired to all other recommendations based on action
            for rec in recommendations:
                if "rule_fired" not in rec:  # Skip if already handled (like pit_stop)
                    rec["rule_fired"] = action_to_rule.get(rec["action"], "unknown")
            
            # Get current position from telemetry fact or gap data
            driver_position = None
            
            # Check telemetry fact
            if 'telemetry' in facts and facts['telemetry'] is not None:
                driver_position = facts['telemetry'].get('position')
            
            # Check gap data
            if driver_position is None and driver_gap_data is not None and not driver_gap_data.empty:
                if 'Position' in driver_gap_data.columns:
                    driver_position = int(driver_gap_data['Position'].iloc[0])
            
            # Default position if still None
            if driver_position is None:
                driver_position = 0
                
            # Get team ID (defaults to 0 if not found)
            team_id = 0
            
            # Try to find team in race_data
            if 'TeamID' in race_data.columns:
                driver_team_data = race_data[race_data['DriverNumber'] == driver_number]
                if not driver_team_data.empty:
                    team_id = int(driver_team_data['TeamID'].iloc[0])
            
            # Extract the compound ID from telemetry fact
            compound_id = None
            if 'telemetry' in facts and facts['telemetry'] is not None:
                compound_id = facts['telemetry'].get('compound_id')
            
            # Add metadata to recommendations
            for rec in recommendations:
                rec['DriverNumber'] = driver_number
                rec['LapNumber'] = lap
                rec['RacePhase'] = facts['race_status']['race_phase'] if 'race_status' in facts else None
                rec['Position'] = driver_position
                rec['Team'] = team_id
                
                # Add tire compound information (NEW)
                rec['CompoundName'] = compound_names.get(compound_id, 'Unknown') if compound_id is not None else 'Unknown'
            
            all_recommendations.extend(recommendations)
            print(f"    Generated {len(recommendations)} recommendations")
            
            # Display which rule systems were activated
            active_systems = engine.active_systems
            active_count = sum(1 for v in active_systems.values() if v)
            if active_count > 0:
                print(f"    Active rule systems: {', '.join(k for k, v in active_systems.items() if v)}")
    
    # Convert to DataFrame
    if all_recommendations:
        results_df = pd.DataFrame(all_recommendations)
        
        # Sort by lap number in ascending order (NEW)
        results_df = results_df.sort_values('LapNumber', ascending=True)
        
        # Reorganize columns to place rule_fired after action (NEW)
        if 'rule_fired' in results_df.columns and 'action' in results_df.columns:
            # Get the current column ordering
            cols = list(results_df.columns)
            
            # Remove rule_fired from its current position
            if 'rule_fired' in cols:
                cols.remove('rule_fired')
                
            # Find the position of action
            action_idx = cols.index('action')
            
            # Insert rule_fired right after action
            cols.insert(action_idx + 1, 'rule_fired')
            
            # Reorder the DataFrame using the new column order
            results_df = results_df[cols]
            
            print("Columns reorganized: 'rule_fired' placed after 'action'")
        
        # Save results
        if output_path:
            results_df.to_csv(output_path, index=False)
            print(f"\nResults saved to: {output_path}")
        
        print(f"\nTotal recommendations: {len(results_df)}")
        return results_df
    else:
        print("No recommendations generated")
        return pd.DataFrame()

### 6.1 Run the analysis for all drivers using the last function 

In [51]:
def run_all_drivers_analysis():
    """
    Run complete analysis with real radios from 2023 Spanish GP.
    
    This function sets up all paths and runs the end-to-end analysis.
    """
    # File paths
    race_data_path = '../../outputs/week3/lap_prediction_data.csv'
    models_path = '../../outputs/week5/models/'
    lap_model_path = '../../outputs/week3/xgb_sequential_model.pkl'
    output_path = '../../outputs/week6/spain_gp_recommendations.csv'
    
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Run analysis (True to extract new radios, False to reuse existing)
    results = analyze_all_drivers_with_real_radios(
        race_data_path=race_data_path,
        models_path=models_path,
        lap_model_path=lap_model_path,
        output_path=output_path,
        extract_new_radios=False  # Change to True to download new data
    )
    
    return results

## ----------------------------------------------------------------------------------
## NOTE: need to verify data passed to the rules in some drivers, further work needed. 
## Detected this event in passing degradation rate to the agent.
## ----------------------------------------------------------------------------------

# Run if directly executed
if __name__ == "__main__":
    results = run_all_drivers_analysis()


ANALYZING ALL DRIVERS WITH REAL RADIOS FROM 2023 SPANISH GP
Started: 2025-06-04 17:24:12
Race data loaded: 1180 rows
Loading existing radio metadata: ../../outputs\week6\radios\spain_2023_radio_metadata.csv

MAPPING RADIO MESSAGES TO RACE LAPS
No date column in race data. Using radio timestamps
Race duration: 153.8 minutes, Total laps: 50
Mapped 126 radios to specific laps

Distribution of radios by lap (first 10 laps):
  • Lap 1: 1 radios
  • Lap 2: 1 radios
  • Lap 3: 3 radios
  • Lap 4: 2 radios
  • Lap 5: 5 radios
  • Lap 17: 1 radios
  • Lap 18: 2 radios
  • Lap 19: 2 radios
  • Lap 20: 2 radios
  • Lap 21: 5 radios
Found 164 processed radio JSON files
Created mapping for 20 drivers with radio data
Data loaded and validated: 1180 rows, 15 columns
Processing Medium tires (ID: 2)...
Processing Hard tires (ID: 3)...
  No laps with new tires for Hard, using TyreAge=2.0 as baseline
Processing Soft tires (ID: 1)...
  No laps with new tires for Soft, using TyreAge=2.0 as baseline
Degrad

c:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\scripts\ML_tyre_pred\ML_utils\N02_model_tire_predictions.py:307: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 0.    -3.478  0.152 -0.098 -0.008  0.174  0.129 -0.527  0.443  0.107
  0.241 -0.023 -0.209  0.255 -0.041 -0.041  0.213 -0.031 -0.135 -0.025
  0.154 -0.163  0.284 -0.097  0.082  3.925]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result_df.loc[group_idx, 'DegradationRate'] = diffs
c:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\scripts\ML_tyre_pred\ML_utils\N02_model_tire_predictions.py:550: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github

Specialized model for compound 2 loaded
Specialized model for compound 3 loaded
Models loaded: 1 global model and 3 specialized models
Prepared tensor for model: shape=torch.Size([339, 5, 16])
Generated ensemble predictions for 339 sequences
Formatted results: 1017 predictions for 20 drivers
Tire predictions generated: 1017 rows
Error loading model: No module named 'xgboost'
Failed to load model. Aborting prediction.
Lap time predictions generated: 0 rows
Loading gap data from: ../../outputs\week6\gap_data.csv
Gap data loaded: 1312 rows
Confirmed total laps from gap data: 66
Strategic analysis points: Laps [6, 11, 12, 16, 17, 21, 22, 25, 26, 27, 30, 31, 32, 35, 36, 37, 40, 41, 42, 45, 46, 47, 50, 51, 52, 55, 56, 57, 60, 61, 62, 65, 66]

--- ANALYZING 20 DRIVERS ---

Analyzing driver #1
  Lap 6/66
  Using radio file: driver_1_radio_20250421_115937.json
  Using gap data from lap 6.0
  Using tire predictions (33 rows) for lap 6

TRANSFORMING FACTS FOR DRIVER #1 AT LAP 6
Driver 1, Lap 6: t

---

### F1 Strategy Rules to Actions Mapping

#### Tire Degradation Rules
- **"high_degradation_pit_stop" rule** → `pit_stop` action  
- **"stint_extension_recommendation" rule** → `extend_stint` action  
- **"early_degradation_warning" rule** → `prepare_pit` action  
- **"predicted_high_degradation_alert" rule** → `consider_pit` action  

#### Lap Time Rules
- **"optimal_performance_window" rule** → `push_strategy` action  
- **"performance_cliff_detection" rule** → `pit_stop` action  
- **"post_traffic_recovery" rule** → `recovery_push` action  

#### NLP Radio Rules
- **"grip_issue_response" rule** → `prioritize_pit` action  
- **"weather_information_adjustment" rule** → `prepare_rain_tires` action  
- **"incident_reaction" rule** → `reevaluate_pit_window` action  

#### Gap Analysis Rules
- **"undercut_opportunity" rule** → `perform_undercut` action  
- **"defensive_pit_stop" rule** → `defensive_pit` action  
- **"strategic_overcut" rule** → `perform_overcut` action  
- **"traffic_management" rule** → `adjust_pit_window` action  


---

#### Quick checking of generated recommendations

In [52]:

# Load the CSV file
recommendations = pd.read_csv('../../outputs/week6/spain_gp_recommendations.csv')

# Count the occurrences of each action type
action_counts = recommendations['action'].value_counts()

print("=== ACTION SUMMARY ===")
print(action_counts)

# Check specific rules by category
print("\n=== GAP RULES ===")
gap_actions = ['perform_undercut', 'defensive_pit', 'perform_overcut', 'adjust_pit_window']
for action in gap_actions:
    count = sum(recommendations['action'] == action)
    status = "✓" if count > 0 else "✗"
    print(f"{status} {action}: {count} occurrences")

print("\n=== LAP TIME RULES ===")
laptime_actions = ['push_strategy', 'pit_stop', 'recovery_push']
for action in laptime_actions:
    count = sum(recommendations['action'] == action)
    status = "✓" if count > 0 else "✗"
    print(f"{status} {action}: {count} occurrences")

print("\n=== DEGRADATION RULES ===")
degradation_actions = ['extend_stint', 'prioritize_pit', 'prepare_pit', 'consider_pit']
for action in degradation_actions:
    count = sum(recommendations['action'] == action)
    status = "✓" if count > 0 else "✗"
    print(f"{status} {action}: {count} occurrences")

print("\n=== RADIO RULES ===")
radio_actions = ['prepare_rain_tires', 'reevaluate_pit_window']
for action in radio_actions:
    count = sum(recommendations['action'] == action)
    status = "✓" if count > 0 else "✗"
    print(f"{status} {action}: {count} occurrences")


=== ACTION SUMMARY ===
action
consider_pit          349
pit_stop              323
extend_stint          256
perform_undercut       79
defensive_pit          74
prepare_rain_tires     33
perform_overcut        24
Name: count, dtype: int64

=== GAP RULES ===
✓ perform_undercut: 79 occurrences
✓ defensive_pit: 74 occurrences
✓ perform_overcut: 24 occurrences
✗ adjust_pit_window: 0 occurrences

=== LAP TIME RULES ===
✗ push_strategy: 0 occurrences
✓ pit_stop: 323 occurrences
✗ recovery_push: 0 occurrences

=== DEGRADATION RULES ===
✓ extend_stint: 256 occurrences
✗ prioritize_pit: 0 occurrences
✗ prepare_pit: 0 occurrences
✓ consider_pit: 349 occurrences

=== RADIO RULES ===
✓ prepare_rain_tires: 33 occurrences
✗ reevaluate_pit_window: 0 occurrences


In [53]:
def test_pit_recommendations_analysis():
    """
    Análisis específico de recomendaciones pit_stop y consider_pit 
    para verificar que tire_age y degradation_rate cambian correctamente
    """
    print("=== ANÁLISIS ESPECÍFICO DE RECOMENDACIONES PIT STOP ===")
    
    # Cargar el CSV de recomendaciones
    csv_path = '../../outputs/week6/spain_gp_recommendations.csv'
    
    try:
        recommendations_df = pd.read_csv(csv_path)
        print(f"✅ CSV cargado: {len(recommendations_df)} recomendaciones totales")
    except FileNotFoundError:
        print(f"❌ Error: No se encontró el archivo {csv_path}")
        return None
    except Exception as e:
        print(f"❌ Error cargando CSV: {str(e)}")
        return None
    
    # Filtrar solo recomendaciones de pit_stop y consider_pit
    pit_actions = ['pit_stop', 'consider_pit']
    pit_recommendations = recommendations_df[recommendations_df['action'].isin(pit_actions)]
    
    print(f"Total recomendaciones de pit: {len(pit_recommendations)}")
    print(f"Acciones encontradas: {pit_recommendations['action'].value_counts().to_dict()}")
    
    if pit_recommendations.empty:
        print("❌ No se encontraron recomendaciones de pit stop")
        return None
    
    # Función para extraer valores de tire_age y degradation_rate de las explicaciones
    def extract_values_from_explanation(explanation):
        import re
        
        tire_age = None
        degradation_rate = None
        
        # Buscar tire age: patrones como "with significant tire age (21 laps)" o "tire age (15 laps)"
        tire_age_match = re.search(r'tire age \((\d+)', explanation)
        if tire_age_match:
            tire_age = int(tire_age_match.group(1))
        
        # Buscar degradation rate: patrones como "degradation rate detected (0.812)" o "degradation (0.315)"
        deg_rate_matches = re.findall(r'degradation.*?[\(\s]([-+]?\d*\.?\d+)', explanation)
        if deg_rate_matches:
            # Tomar el primer valor numérico que parezca una tasa de degradación
            for match in deg_rate_matches:
                try:
                    rate = float(match)
                    if -5.0 <= rate <= 5.0:  # Rango razonable para degradation rate
                        degradation_rate = rate
                        break
                except:
                    continue
        
        return tire_age, degradation_rate
    
    # Probar con varios conductores que tengan recomendaciones de pit
    drivers_with_pits = pit_recommendations['DriverNumber'].unique()
    print(f"Conductores con recomendaciones de pit: {sorted(drivers_with_pits)}")
    
    # Analizar cada conductor
    for test_driver in sorted(drivers_with_pits)[:3]:  # Analizar los primeros 3 conductores
        print(f"\n{'='*70}")
        print(f"ANÁLISIS DETALLADO - CONDUCTOR {test_driver}")
        print(f"{'='*70}")
        
        driver_pits = pit_recommendations[pit_recommendations['DriverNumber'] == test_driver].sort_values('LapNumber')
        
        print(f"Recomendaciones de pit para conductor {test_driver}: {len(driver_pits)}")
        print(f"Vueltas con recomendaciones: {driver_pits['LapNumber'].tolist()}")
        
        if len(driver_pits) < 2:
            print("⚠️  Solo 1 recomendación de pit, no se puede verificar cambio")
            continue
        
        # Analizar cada recomendación
        pit_analysis = []
        
        for _, rec in driver_pits.iterrows():
            tire_age, degradation_rate = extract_values_from_explanation(rec['explanation'])
            
            analysis = {
                'lap': rec['LapNumber'],
                'action': rec['action'],
                'rule_fired': rec['rule_fired'],
                'confidence': rec['confidence'],
                'priority': rec['priority'],
                'tire_age': tire_age,
                'degradation_rate': degradation_rate,
                'explanation': rec['explanation']
            }
            pit_analysis.append(analysis)
            
            print(f"\n--- VUELTA {rec['LapNumber']} ---")
            print(f"Acción: {rec['action']}")
            print(f"Regla: {rec['rule_fired']}")
            print(f"Tire age extraído: {tire_age}")
            print(f"Degradation rate extraído: {degradation_rate}")
            print(f"Confianza: {rec['confidence']:.2f}")
            print(f"Explicación: {rec['explanation']}")
        
        # Verificar cambios en los valores
        print(f"\n{'='*50}")
        print(f"VERIFICACIÓN DE CAMBIOS - CONDUCTOR {test_driver}")
        print(f"{'='*50}")
        
        # Extraer valores para análisis
        tire_ages = [a['tire_age'] for a in pit_analysis if a['tire_age'] is not None]
        degradation_rates = [a['degradation_rate'] for a in pit_analysis if a['degradation_rate'] is not None]
        
        print(f"Tire ages extraídos: {tire_ages}")
        print(f"Degradation rates extraídos: {degradation_rates}")
        
        # Verificar cambios en tire_age
        if len(tire_ages) > 1:
            if len(set(tire_ages)) > 1:
                print("✅ TIRE AGES CAMBIAN CORRECTAMENTE")
                print(f"   Progresión: {' → '.join(map(str, tire_ages))}")
            else:
                print("❌ TIRE AGES NO CAMBIAN (todos iguales)")
                print(f"   Valor constante: {tire_ages[0]}")
        else:
            print("⚠️  Insuficientes tire_ages extraídos para verificar cambio")
        
        # Verificar cambios en degradation_rate
        if len(degradation_rates) > 1:
            if len(set(degradation_rates)) > 1:
                print("✅ DEGRADATION RATES CAMBIAN CORRECTAMENTE")
                print(f"   Progresión: {' → '.join(f'{d:.3f}' for d in degradation_rates)}")
            else:
                print("❌ DEGRADATION RATES NO CAMBIAN (todos iguales)")
                print(f"   Valor constante: {degradation_rates[0]:.3f}")
        else:
            print("⚠️  Insuficientes degradation_rates extraídos para verificar cambio")
        
        # Mostrar tabla resumen
        print(f"\nTabla resumen:")
        print(f"{'Vuelta':<8}{'Action':<15}{'Tire Age':<10}{'Deg Rate':<12}{'Rule':<25}")
        print("-" * 70)
        for analysis in pit_analysis:
            tire_age_str = str(analysis['tire_age']) if analysis['tire_age'] is not None else 'N/A'
            deg_rate_str = f"{analysis['degradation_rate']:.3f}" if analysis['degradation_rate'] is not None else 'N/A'
            rule_short = analysis['rule_fired'][:22] + '...' if len(analysis['rule_fired']) > 25 else analysis['rule_fired']
            print(f"{analysis['lap']:<8}{analysis['action']:<15}{tire_age_str:<10}{deg_rate_str:<12}{rule_short:<25}")

def analyze_pit_patterns_across_all_drivers():
    """
    Análisis de patrones de pit stops en todos los conductores
    """
    print(f"\n{'='*80}")
    print("ANÁLISIS DE PATRONES DE PIT STOP - TODOS LOS CONDUCTORES")
    print(f"{'='*80}")
    
    # Cargar CSV
    csv_path = '../../outputs/week6/spain_gp_recommendations.csv'
    try:
        recommendations_df = pd.read_csv(csv_path)
    except:
        print("❌ Error cargando CSV")
        return
    
    # Filtrar recomendaciones de pit
    pit_actions = ['pit_stop', 'consider_pit']
    pit_recommendations = recommendations_df[recommendations_df['action'].isin(pit_actions)]
    
    print(f"Total recomendaciones de pit en todo el dataset: {len(pit_recommendations)}")
    
    # Análisis por conductor
    pit_by_driver = pit_recommendations.groupby('DriverNumber').agg({
        'LapNumber': ['count', 'min', 'max', list],
        'action': lambda x: x.value_counts().to_dict(),
        'rule_fired': lambda x: x.value_counts().to_dict()
    }).round(3)
    
    print(f"\nRecomendaciones de pit por conductor:")
    print("-" * 80)
    
    for driver in sorted(pit_recommendations['DriverNumber'].unique()):
        driver_pits = pit_recommendations[pit_recommendations['DriverNumber'] == driver]
        laps = sorted(driver_pits['LapNumber'].tolist())
        actions = driver_pits['action'].value_counts().to_dict()
        rules = driver_pits['rule_fired'].value_counts().to_dict()
        
        print(f"Conductor {driver:2d}: {len(driver_pits):2d} recomendaciones en vueltas {laps}")
        print(f"            Acciones: {actions}")
        print(f"            Reglas:   {rules}")
        print()
    
    # Análisis temporal
    print(f"Distribución de recomendaciones de pit por vuelta:")
    lap_distribution = pit_recommendations['LapNumber'].value_counts().sort_index()
    print(lap_distribution.head(10))
    
    # Análisis por regla
    print(f"\nDistribución por regla activada:")
    rule_distribution = pit_recommendations['rule_fired'].value_counts()
    for rule, count in rule_distribution.items():
        print(f"{rule}: {count} veces")

def extract_and_verify_pit_values():
    """
    Extracción y verificación específica de valores de tire_age y degradation_rate 
    en recomendaciones de pit para confirmar que cambian
    """
    print(f"\n{'='*80}")
    print("EXTRACCIÓN Y VERIFICACIÓN DE VALORES EN RECOMENDACIONES PIT")
    print(f"{'='*80}")
    
    # Cargar CSV
    csv_path = '../../outputs/week6/spain_gp_recommendations.csv'
    try:
        recommendations_df = pd.read_csv(csv_path)
    except:
        print("❌ Error cargando CSV")
        return
    
    # Filtrar pit_stop y consider_pit
    pit_recommendations = recommendations_df[
        recommendations_df['action'].isin(['pit_stop', 'consider_pit'])
    ].copy()
    
    print(f"Analizando {len(pit_recommendations)} recomendaciones de pit...")
    
    # Función mejorada para extraer valores
    def extract_all_values(explanation):
        import re
        
        # Buscar tire age con diferentes patrones
        tire_age_patterns = [
            r'tire age \((\d+)',
            r'age \((\d+)',
            r'with (\d+) laps',
            r'despite tire age \((\d+)',
            r'significant tire age \((\d+)'
        ]
        
        tire_age = None
        for pattern in tire_age_patterns:
            match = re.search(pattern, explanation)
            if match:
                tire_age = int(match.group(1))
                break
        
        # Buscar degradation rate con diferentes patrones
        deg_patterns = [
            r'degradation rate detected \(([-+]?\d*\.?\d+)',
            r'degradation \(([-+]?\d*\.?\d+)',
            r'rate: ([-+]?\d*\.?\d+)',
            r'predicts ([-+]?\d*\.?\d+)',
            r'Model predicts ([-+]?\d*\.?\d+)'
        ]
        
        degradation_rate = None
        for pattern in deg_patterns:
            matches = re.findall(pattern, explanation)
            if matches:
                try:
                    rate = float(matches[0])
                    if -10.0 <= rate <= 10.0:  # Rango más amplio
                        degradation_rate = rate
                        break
                except:
                    continue
        
        return tire_age, degradation_rate
    
    # Extraer valores para todas las recomendaciones
    extracted_data = []
    for _, rec in pit_recommendations.iterrows():
        tire_age, degradation_rate = extract_all_values(rec['explanation'])
        extracted_data.append({
            'DriverNumber': rec['DriverNumber'],
            'LapNumber': rec['LapNumber'],
            'action': rec['action'],
            'rule_fired': rec['rule_fired'],
            'tire_age': tire_age,
            'degradation_rate': degradation_rate,
            'explanation': rec['explanation']
        })
    
    extracted_df = pd.DataFrame(extracted_data)
    
    # Mostrar estadísticas de extracción
    tire_age_extracted = extracted_df['tire_age'].notna().sum()
    deg_rate_extracted = extracted_df['degradation_rate'].notna().sum()
    
    print(f"Valores extraídos:")
    print(f"  Tire age: {tire_age_extracted}/{len(extracted_df)} ({tire_age_extracted/len(extracted_df)*100:.1f}%)")
    print(f"  Degradation rate: {deg_rate_extracted}/{len(extracted_df)} ({deg_rate_extracted/len(extracted_df)*100:.1f}%)")
    
    # Análisis de cambios por conductor
    print(f"\nAnálisis de cambios por conductor:")
    print("-" * 60)
    
    for driver in sorted(extracted_df['DriverNumber'].unique()):
        driver_data = extracted_df[extracted_df['DriverNumber'] == driver].sort_values('LapNumber')
        
        if len(driver_data) < 2:
            continue
        
        tire_ages = driver_data['tire_age'].dropna().tolist()
        deg_rates = driver_data['degradation_rate'].dropna().tolist()
        
        print(f"Conductor {driver}:")
        print(f"  Vueltas: {driver_data['LapNumber'].tolist()}")
        
        if len(tire_ages) > 1:
            tire_change = "✅ CAMBIAN" if len(set(tire_ages)) > 1 else "❌ NO CAMBIAN"
            print(f"  Tire ages: {tire_ages} → {tire_change}")
        else:
            print(f"  Tire ages: {tire_ages} → ⚠️ INSUFICIENTES")
        
        if len(deg_rates) > 1:
            deg_change = "✅ CAMBIAN" if len(set(deg_rates)) > 1 else "❌ NO CAMBIAN"
            print(f"  Deg rates: {[f'{d:.3f}' for d in deg_rates]} → {deg_change}")
        else:
            print(f"  Deg rates: {[f'{d:.3f}' for d in deg_rates]} → ⚠️ INSUFICIENTES")
        
        print()
    
    return extracted_df

# Ejecutar análisis completo
print("=== EJECUTANDO ANÁLISIS COMPLETO DE PIT STOPS ===\n")

# 1. Análisis específico de pit stops
pit_results = test_pit_recommendations_analysis()

# 2. Análisis de patrones generales
analyze_pit_patterns_across_all_drivers()

# 3. Extracción y verificación detallada
extracted_values = extract_and_verify_pit_values()

=== EJECUTANDO ANÁLISIS COMPLETO DE PIT STOPS ===

=== ANÁLISIS ESPECÍFICO DE RECOMENDACIONES PIT STOP ===
✅ CSV cargado: 1138 recomendaciones totales
Total recomendaciones de pit: 672
Acciones encontradas: {'consider_pit': 349, 'pit_stop': 323}
Conductores con recomendaciones de pit: [np.int64(1), np.int64(2), np.int64(4), np.int64(10), np.int64(11), np.int64(14), np.int64(16), np.int64(18), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(27), np.int64(31), np.int64(44), np.int64(55), np.int64(63), np.int64(77), np.int64(81)]

ANÁLISIS DETALLADO - CONDUCTOR 1
Recomendaciones de pit para conductor 1: 60
Vueltas con recomendaciones: [6, 6, 11, 11, 12, 12, 16, 16, 17, 17, 21, 21, 27, 27, 30, 30, 31, 31, 32, 32, 35, 35, 36, 36, 37, 37, 40, 40, 41, 41, 42, 42, 45, 45, 46, 46, 47, 47, 50, 50, 51, 51, 52, 52, 55, 55, 56, 56, 57, 57, 60, 60, 61, 61, 62, 62, 65, 65, 66, 66]

--- VUELTA 6 ---
Acción: pit_stop
Regla: high_degradation_pit_stop
Tire age extraído: 26
Degradation ra